In [1]:
import tweepy as tw
import pandas as pd
import datetime as dt
import fastparquet 
from log_module import logger_error,logger_info

In [2]:
consumer_key = '1m0fGzCVJPfKCXOyPEoNPs9fy'
consumer_secret = 'Ze8nzsoHuMOWRCgIRu6pnvufsqq912G5rlTCRGJL75EmmSzBWq'
access_token = '74554547-FOA0hIvS8qS2RbPFvEYvk0CmYfRpXzsKpAi9QxbuZ'
access_token_secret = 'Nm83fui8LlDrsdrStvpmOeIUA7T8YbyWgUiYl1i2ylS9b'

auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth,wait_on_rate_limit=True)


In [14]:
# Define the search term 
search_words = "#FlixBus"

# Define the date range to look for the search_word
# if both are set to 0 = current day
start_days = 1
end_days = 0
today = dt.date.today()

date_since = today - dt.timedelta(days=start_days)
date_until = today + dt.timedelta(days=end_days)
print(date_since,date_until)

2021-03-01 2021-03-02


In [15]:
# Connecting with API to do a search based on date and query
try:
    tweets = tw.Cursor(api.search 
                            ,q = search_words
                            ,since = date_since
                            ,until = date_until
                        ).items(200)

    users_locs = [[ 
                     tweet.id
                    ,tweet.user.id
                    ,tweet.user.location
                    ,tweet.user.followers_count
                    ,tweet.created_at
                    ,tweet.text 
                    ,tweet.user.statuses_count
                    ]for tweet in tweets]
    logger_info(f"Hashtag search was sucessfull from {date_since} to {date_until} !",today)
except Exception as e:
    logger_error(e,today)

In [16]:
users_locs

[[1366499320766357506,
  1331037722086158338,
  'Metaverse',
  885,
  datetime.datetime(2021, 3, 1, 21, 23, 21),
  'RT @NCrypted: How does #Flixbus make money as a #Smartbus #travel platform?  https://t.co/nVJVcClL5C #busservice #FlixMobility #startups #e…',
  93002],
 [1366498478445236238,
  199576549,
  'sales@ncrypted.com',
  10191,
  datetime.datetime(2021, 3, 1, 21, 20),
  'How does #Flixbus make money as a #Smartbus #travel platform?  https://t.co/nVJVcClL5C #busservice #FlixMobility… https://t.co/mV4ipYPoed',
  5624]]

In [17]:
# Creating parquet file and storing in the raw zone
try:
    if users_locs:
        df = pd.DataFrame(users_locs,columns=[
                                                'tweetId'
                                                ,'userId'
                                                ,'location'
                                                ,'userFollowers'
                                                ,'createdAt'
                                                ,'tweetText'
                                                ,'userTweets'])
        df.to_parquet(f'rawZone/flixBus#{today}.parquet.gzip', compression='gzip')
        logger_info(f"Hashtag search generated file flixBus#{today} from {date_since} to {date_until} !",today)
    else:
        logger_info(f"Hashtag search generated no data from {date_since} to {date_until} !",today)
except Exception as e:
    logger_error(e,today)
    